# SD202 TP2 - Normalization and SQL

The objectives of this TP are the following:

1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this lab, we are going to use a database containing wine information related to 'production' and 'sales'. 

Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


We need to prepare the SQL environment:

In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


We recommend inline __%sql__ as an alternative to sqlite3 package

In [4]:
%load_ext sql
%sql sqlite:///wine.db

Now, we can see the content of the tables using SQL queries:

In [5]:
%sql SELECT DISTINCT NV, CRU, MILL, DEGRE FROM MASTER1;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
None,None,None,None
1,Mercurey,1980,11.5
2,Julienas,1974,11.3
3,Savigny les Beaunes,1978,12.1
4,Mercurey,1980,10.9
5,Pommard,1976,11.7
6,Mercurey,1981,11.2
7,Grands Echezeaux,1968,11.7
8,Cotes de Beaune Villages,1975,12.3
9,Chapelle Chambertin,1973,11.9


# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables Master1 and Master2 are in the First Normal Form (1NF) and suffer from data redundancy. 

__1.1__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NP -> NOM, PRENOM, REGION

NV, NP -> QTE

Convert table Master1 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.

* Explain your answer
* Describe the schema of new tables and how they relate

To convert the "Master1" table to the Second Normal Form (2NF) or Third Normal Form (3NF) based on the given set of functional dependencies (FDs), we need to analyze the dependencies and split the table accordingly. Let's examine the FDs and proceed with the normalization process:

Based on the given FDs, it appears that the primary key for the "Master1" table is a composite key consisting of NV and NP since they appear on the right side of the FDs.

Based on the FDs, we can identify the following tables:

Table 1: Wine
Attributes: NV (Primary Key), CRU, DEGRE, MILL

Table 2: Producer
Attributes: NP (Primary Key), NOM, PRENOM, REGION

Table 3: Quantity
Attributes: NV (Foreign Key), NP (Foreign Key), QTE

In Table 3 (Quantity), both NV and NP are foreign keys referencing the primary keys of Table 1 (Wine) and Table 2 (Producer), respectively. This establishes the relationships between the tables.

Table Schema Summary:

Table: Wine
| Attribute | Description     |
| --------- | --------------- |
| NV        | Wine number (PK) |
| CRU       | Vineyard        |
| DEGRE     | Alcohol content |
| MILL      | Vintage year    |

Table: Producer
| Attribute | Description           |
| --------- | --------------------- |
| NP        | Producer number (PK)  |
| NOM       | Producer's last name  |
| PRENOM    | Producer's first name |
| REGION    | Production region     |

Table: Quantity
| Attribute | Description         |
| --------- | ------------------- |
| NV        | Wine number (FK)    |
| NP        | Producer number (FK)|
| QTE       | Number of bottles   |

In the normalized schema, each table represents a separate entity, and the relationships between the tables are maintained through the use of foreign keys. This ensures data integrity and eliminates redundant information.

The given FDs do not indicate any dependencies, so 2NF is not required, and the tables are already in 3NF.

__1.2__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NB -> NOM, PRENOM, TYPE

NV, NB -> DATES, LIEU, QTE

LIEU -> REGION 

Convert table Master2 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable. 
* Explain your answer
* Describe the schema of new tables and how they relate

To convert the "Master2" table to the Second Normal Form (2NF) or Third Normal Form (3NF) based on the given set of functional dependencies (FDs), let's analyze the dependencies and proceed with the normalization process.

Based on the given FDs, it appears that the primary key for the "Master2" table is a composite key consisting of NV and NB since they appear on the right side of the FDs.

Based on the FDs, we can identify the following tables:

Table 1: Wine
Attributes: NV (Primary Key), CRU, DEGRE, MILL

Table 2: Client
Attributes: NB (Primary Key), NOM, PRENOM, TYPE

Table 3: Purchase
Attributes: NV (Foreign Key), NB (Foreign Key), DATES, LIEU, QTE

Table 4: Region
Attributes: LIEU (Primary Key), REGION

In Table 3 (Purchase), both NV and NB are foreign keys referencing the primary keys of Table 1 (Wine) and Table 2 (Client), respectively. This establishes the relationships between the tables.

In Table 4 (Region), LIEU is the primary key, and it serves as a foreign key referencing the LIEU attribute in Table 3 (Purchase).

Table Schema Summary:

Table: Wine
| Attribute | Description     |
| --------- | --------------- |
| NV        | Wine number (PK) |
| CRU       | Vineyard        |
| DEGRE     | Alcohol content |
| MILL      | Vintage year    |

Table: Client
| Attribute | Description           |
| --------- | --------------------- |
| NB        | Client number (PK)    |
| NOM       | Client's last name    |
| PRENOM    | Client's first name   |
| TYPE      | Type of client        |

Table: Purchase
| Attribute | Description            |
| --------- | ---------------------- |
| NV        | Wine number (FK)       |
| NB        | Client number (FK)     |
| DATES     | Buying date            |
| LIEU      | Place where wine sold  |
| QTE       | Number of bottles      |

Table: Region
| Attribute | Description                      |
| --------- | -------------------------------- |
| LIEU      | Place where wine was sold (PK)   |
| REGION    | Administrative Region description|

In the normalized schema, each table represents a separate entity, and the relationships between the tables are maintained through the use of foreign keys. This ensures data integrity and eliminates redundant information.

The given FDs do not indicate any dependencies, so 2NF is not required, and the tables are already in 3NF.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables. A table can be created from the result of a query. In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [6]:
%sql DROP TABLE IF EXISTS dummy;

# Create dummy table
%sql CREATE TABLE dummy AS \
SELECT DISTINCT DEGRE \
FROM MASTER1;

print("\nContent of the database")
printSchema(conn)

print("\nContent of dummy")
%sql SELECT * FROM dummy

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.

Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)

Content of dummy
 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


In [7]:
# Remove dummy table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

__1.3__ Create the new tables from Master1:

In [13]:
# Wine
%sql DROP TABLE IF EXISTS Wine1;

%sql CREATE TABLE Wine1 AS \
SELECT NV, CRU, DEGRE, MILL \
FROM Master1;

# Producer
%sql DROP TABLE IF EXISTS Producer;

%sql CREATE TABLE Producer AS \
SELECT NP, NOM, PRENOM, REGION \
FROM Master1;

# Quantity
%sql DROP TABLE IF EXISTS Quantity;

%sql CREATE TABLE Quantity AS \
SELECT NV, NP, QTE \
FROM Master1;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.


[]

__1.4__ Create the new tables from Master2:

In [14]:
# Wine
%sql DROP TABLE IF EXISTS Wine2;

%sql CREATE TABLE Wine2 AS \
SELECT NV, CRU, DEGRE, MILL \
FROM Master2;

# Client
%sql DROP TABLE IF EXISTS Client;

%sql CREATE TABLE Client AS \
SELECT NB, NOM, PRENOM, TYPE \
FROM Master2;

# Purchase
%sql DROP TABLE IF EXISTS Purchase;

%sql CREATE TABLE Purchase AS \
SELECT NV, NB, DATES, LIEU, QTE \
FROM Master2;

# Region
%sql DROP TABLE IF EXISTS Region;

%sql CREATE TABLE Region AS \
SELECT LIEU, REGION \
FROM Master2;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

__2.1__ What are the different types of clients (buveurs) by volume of purchases?

In [10]:
%sql SELECT TYPE, COUNT(*) AS TotalClients FROM Client GROUP BY TYPE;

 * sqlite:///wine.db
Done.


TYPE,TotalClients
None,55
gros,37
moyen,26
petit,67


__2.2__ What regions produce Pommard or Brouilly?

In [16]:
%sql SELECT DISTINCT REGION FROM Region WHERE LIEU IN (SELECT DISTINCT LIEU FROM Wine2 WHERE CRU IN ('Pommard', 'Brouilly'));

 * sqlite:///wine.db
Done.


REGION
NOUVELLE-AQUITAINE
ÎLE-DE-FRANCE
BRETAGNE
AUVERGNE-RHÔNE-ALPES
PROVENCE-ALPES-CÔTE D'AZUR
BOURGOGNE-FRANCHE-COMTÉ
OCCITANIE
HAUTS-DE-FRANCE
GRAND EST
NORMANDIE


__2.3__ What regions produce Pommard and Brouilly?

In [18]:
%sql SELECT DISTINCT Region.REGION \
FROM Wine2 \
JOIN Purchase ON Wine2.NV = Purchase.NV \
JOIN Region ON Purchase.LIEU = Region.LIEU \
WHERE Wine2.CRU IN ('Pommard', 'Brouilly');


 * sqlite:///wine.db
Done.


REGION
HAUTS-DE-FRANCE
BOURGOGNE-FRANCHE-COMTÉ


__2.4__ Get the number of wines bought by CRU and Millésime

In [19]:
%sql SELECT Wine2.CRU, Wine2.MILL, SUM(Purchase.QTE) AS TotalQuantity \
FROM Wine2 \
JOIN Purchase ON Wine2.NV = Purchase.NV \
GROUP BY Wine2.CRU, Wine2.MILL;

 * sqlite:///wine.db
Done.


CRU,MILL,TotalQuantity
Arbois,1976,None
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,14
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,1560
Bellet,1976,None
Blanquette de Limoux,1978,None


__2.5__ Retrieve the wine number (NV) of wines produced by more than three producers

In [22]:
%sql SELECT NV \
FROM Quantity \
GROUP BY NV \
HAVING COUNT(DISTINCT NP) > 3;


 * sqlite:///wine.db
Done.


NV
None
45
78
89
98


__2.6__ Which producers have not produced any wine?

Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


__2.7__ What clients (buveurs) have bought at least one wine from 1980?

Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


__2.8__ What clients (buveurs) have NOT bought any wine from 1980?

Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


__2.9__ What clients (buveurs) have bought ONLY wines from 1980?

Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


__2.10__ List all wines from 1980

Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


__2.11__ What are the wines from 1980 bought by NB=2?

Done.


NV,NB,DATES,LIEU,QTE,CRU,MILL,DEGRE
1,2,1977-11-02,BORDEAUX,33,Mercurey,1980,11.5


__2.12__ What clients (buveurs) have bought ALL the wines from 1980?

Done.


NB,NOM,PRENOM
44,Gide,Andre
